## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)
# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Camana,PE,-16.62,-72.71,59.86,82,88,8.66,overcast clouds
1,1,Atuona,PF,-9.80,-139.03,78.37,77,16,14.97,few clouds
2,2,Bose,IT,45.46,8.01,66.99,78,95,5.55,overcast clouds
3,3,Hofn,IS,64.25,-15.21,44.60,93,90,3.36,overcast clouds
4,4,Houma,US,29.60,-90.72,80.01,83,1,3.36,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

Unnamed: 0             309
City                   309
Country                308
Lat                    309
Lng                    309
Max Temp               309
Humidity               309
Cloudiness             309
Wind Speed             309
Current Description    309
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.37,few clouds,-9.80,-139.03,
2,Bose,IT,66.99,overcast clouds,45.46,8.01,
4,Houma,US,80.01,clear sky,29.60,-90.72,
7,Padang,ID,83.73,broken clouds,-0.95,100.35,
8,Hilo,US,84.20,overcast clouds,19.73,-155.09,
9,Babakan,ID,80.89,clear sky,-7.28,107.22,
10,Rabat,MA,73.40,broken clouds,33.99,-6.84,
11,Yaan,NG,68.07,overcast clouds,7.38,8.57,
12,Kodiak,US,66.20,overcast clouds,57.79,-152.41,
18,Souillac,MU,69.80,scattered clouds,-20.52,57.52,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city")
        

Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping the city
Hotel not found...skipping t

In [10]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df["Hotel Name"].replace('',np.nan,inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   278
Country                278
Max Temp               278
Current Description    278
Lat                    278
Lng                    278
Hotel Name             278
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))